# Model Training
This script trains the model to predict the marker positions.

In [1]:
import pandas as pd
import numpy as np

import math

from sklearn.utils import shuffle 

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.initializers import glorot_uniform
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau
from keras import metrics

import h5py

import datetime
import time

from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger

import random

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Preparing training and test set

In [2]:
HDF5_PATH = "data.hdf5"

In [3]:
hdf = h5py.File(HDF5_PATH, "r")

In [4]:
train_x = hdf["train/data"]
train_y = hdf["train/labels"]

test_x = hdf["test/data"]
test_y = hdf["test/labels"]

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(2083884, 21, 3) (2083884, 63) (534252, 21, 3) (534252, 63)


In [5]:
startTime = str(round(time.time() * 1000))

In [6]:
def log(s):
    with open("status_Training_imageSingleHandNoR." + startTime + ".txt", "a") as myfile:
        myfile.write("[" + str(datetime.datetime.now()) + "]," + s + "\n")

# Training

In [7]:
def addRandomNoise(gs, ls, batch_size):
    for i in range(0, batch_size):
        random_array = np.random.randint(-2,3,(63))
        gs[i] = gs[i] + random_array
        ls[i] = ls[i] + (random_array / 1000)
        
    return {'gs':gs, 'ls':ls}

In [8]:
def removeMarkersRandomly(coordinates):
    for hand in range(0, len(coordinates)):
        numOfMarkersToRemove = random.randint(0, 4)
        for i in range(0, numOfMarkersToRemove):
            indexToRemove = random.randint(0, (len(coordinates[hand]) - 1))
            coordinates[hand].pop(indexToRemove)
            
    return coordinates

In [9]:
def addGhostMarkers(coordinates):
    for hand in range(0, len(coordinates)):
        numOfMarkersToAdd = random.randint(0, 4)
        for i in range(0, numOfMarkersToAdd):
            x = random.randint(0, nop - 1)
            z = random.randint(0, nop - 1)
            y = random.randint(-ultimateY, ultimateY)
            coordinates[hand].append([x,y,z])
            
    return coordinates

In [10]:
def myGeneratorImage(set_name, batch_size):
    hdf = h5py.File(HDF5_PATH, "r")
    # in mm
    pData = hdf[set_name + "/data"]
    # in m
    pLabels = hdf[set_name + "/labels"]
    
    len_train = pData.shape[0]
    randomBatchOrder = np.arange(len_train // batch_size)    
    
       
    while True:
        np.random.shuffle(randomBatchOrder) 
        
        for i in range(len_train // batch_size):
            idx = randomBatchOrder[i]
            
            # read data and labels for current batch
            gs = pData[idx * batch_size: (idx+1) * batch_size]
            ls = pLabels[idx * batch_size: (idx+1) * batch_size]
            
            gs = np.array(gs)
            gs = gs.reshape(batch_size, 63)
            ls = np.array(ls)
            
            noisy = addRandomNoise(gs, ls, batch_size)
            gs = noisy['gs']
            gs = gs.reshape(-1, 21, 3)
            ls = noisy['ls']
            ls = ls * 1000

            
            # remove y from ls
            ls = ls.reshape(-1)
            ls = list(ls)
            del ls[1::3]
            ls = np.array(ls)
            ls = ls.reshape(-1,21 * 2)
            
            
            gs = createImageSingle(gs)

            shuffled = shuffle(gs, ls)
            yield shuffled[0], shuffled[1]

In [11]:
resolutionPcm = 4
resolutionPmm = resolutionPcm / 10

# image size in cm
imageSize = 25
# max y in mm
ultimateY = 120

# number of pixels
nop = imageSize * resolutionPcm
zzz = [nop / 2, 0, nop - (8 * resolutionPcm)]

def createImageSingle(coordinates): # coordinates in mm
    images = np.zeros((coordinates.shape[0], nop, nop, 1))
    
    coordinates = (coordinates * resolutionPmm) + zzz
    
    # take care that values fit to image size
    coordinates = coordinates.reshape(-1)
    xt = coordinates[0::3]
    yt = coordinates[1::3]
    zt = coordinates[2::3]
    xt[np.where(xt < 0)] = 0
    zt[np.where(zt < 0)] = 0
    xt[np.where(xt >= nop)] = nop - 1
    zt[np.where(zt >= nop)] = nop - 1

    # remove resolutionPmm from y again
    yt = (yt / resolutionPmm)
    # set max value for y to ultimateY
    yt[np.where(yt > ultimateY)] = ultimateY
    yt[np.where(yt < -ultimateY)] = -ultimateY
    # normalize yt
    yt = (yt / ultimateY)
    coordinates[0::3] = xt
    coordinates[1::3] = yt
    coordinates[2::3] = zt
    coordinates = coordinates.reshape(-1, 21, 3)
    
    coordinates = coordinates.tolist()
    # remove markers randomly
    coordinates = removeMarkersRandomly(coordinates)
    # add random ghost markers
    coordinates = addGhostMarkers(coordinates)
    
    
    # coordinates as float
    coordinatesF = coordinates
    # coordinates as int
    coordinates = [[[int(z) for z in y] for y in x] for x in coordinates]
    for i in range(0, len(coordinates)):
        coords = coordinates[i]
        coordsF = coordinatesF[i]
        
        for j in range(0, len(coords)):
            # xz-plane image with y value
            images[i][coords[j][0]][coords[j][2]][0] = coordsF[j][1]# y values are normalized
            
    return images

In [12]:
class My_Callback(keras.callbacks.Callback):
    def on_train_end(self, logs={}):
        log("Training ended")
        return
 
    def on_epoch_end(self, epoch, logs={}):
        log("Ending Epoch")
        log(str(logs))
        return

In [13]:
batch_size = 150
num_output = 21 * 2 # (x,z) for 21 markers! No R_Shapes!
epochs = 10000 


# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = (train_x, train_y), (test_x, test_y)


###########################################
## GPU training configuration
###########################################
config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True, device_count = {'GPU' : 4})
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction=1
config.gpu_options.allocator_type = 'BFC'

In [ ]:
with tf.device('/gpu:0'):
    session = tf.Session(config=config)
    K.set_session(session)
    input_shape = (nop, nop, 1)

    ###########################################
    ## Model architecture 
    ###########################################
    model = Sequential()

    # First layer of convolution and max pooling
    model.add(Conv2D(filters = 64, kernel_size = (6,6),padding = 'Same', 
                     activation ='relu', input_shape = input_shape))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPool2D(pool_size=(4,4)))
    model.add(Dropout(0.5))
    
    # Second layer of convolution and max pooling
    model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.5))

    # Dense layer
    model.add(Flatten())
    model.add(Dense(512, activation = "relu", kernel_initializer=glorot_uniform()))
    model.add(BatchNormalization(axis=-1))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation = "relu", kernel_initializer=glorot_uniform()))
    model.add(BatchNormalization(axis=-1))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation = "relu", kernel_initializer=glorot_uniform()))
    model.add(BatchNormalization(axis=-1))
    model.add(Dropout(0.5))
    
    # Output layer 
    model.add(Dense(num_output)) 
    
    ###########################################
    ## Model training 
    ###########################################     
    def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
        
    optimizer = keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss=rmse,
                  optimizer=optimizer,
                  metrics=[rmse])

    learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0000001)
    
    filepath="weights.best.imageSingleHandNoR." + startTime + ".hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
    
    customCB = My_Callback()
    model.fit_generator(myGeneratorImage("train", batch_size),
                        steps_per_epoch=len(train_x) // batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=myGeneratorImage("test", batch_size),
                        validation_steps=len(test_x) // batch_size,
                        callbacks=[checkpoint, customCB, learning_rate_reduction])

Epoch 1/10000
16029/16029 [==============================] - 6787s 423ms/step - loss: 64.8719 - rmse: 64.8719 - val_loss: 58.9132 - val_rmse: 58.9132

Epoch 00001: val_loss improved from inf to 58.91318, saving model to weights.best.imageSingleHandNoR.1542827622775.hdf5
Epoch 2/10000
16029/16029 [==============================] - 6774s 423ms/step - loss: 55.5422 - rmse: 55.5422 - val_loss: 44.5851 - val_rmse: 44.5851

Epoch 00002: val_loss improved from 58.91318 to 44.58513, saving model to weights.best.imageSingleHandNoR.1542827622775.hdf5
Epoch 3/10000
  173/16029 [..............................] - ETA: 1:40:57 - loss: 48.1495 - rmse: 48.1495